# QKRA-ASSL

This file performs 1 iteration of RWTA for X values in a range specified in the central file : 
* Link resources are initialized
* CRs are generated
* Priority queue is generated
* The shortest feasible path of the K-SP is chosen
* Resourcess are allocated based on FF approach

Performance metrics(NSP, TUR, SRCR) is calculated

In [ ]:
while step < steps:    # Loop over various X values, incrementing by a fixed factor

    Links.initialize_links(dir_edges)    # Initialize the link resources
    CRs = CR.generate_crs(X)    # Generating X Crs
    PQ = CR.create_priority_queue()    # Creating the priority Queue
        
    for cr in PQ:                                # Allocating Resources on priority basis  
        candidate_path = k_SP[cr.s][cr.d]                           
        Links.FF(candidate_path, cr, aslc)        # Attempting to allocate resources; will return True if resources have been allocated, otherwise False

    # Capture the output of the functions
    with suppress_stdout():
        srcr[z, step] = CR.SRCR(CRs)
        tur[z, step] = Links.TUR()
        nsp[z, step] = CR.NSP(CRs)
    
    # SRCR[z, step] = CR.Allocated/X
    # TSUR[z, step] = Links.UTS/Links.total_ts
    # NSP[z, step] = CR.Nsp

    CR_desc[z, step] = {cr.index : (cr.sl, cr.status) for cr in PQ}    # This will need to be corrected in case the improvisation is used

    step += 1
    X += 25    # Incrementing X by a factor
    pbar.update(1)

################################ Scaling ##########################
Srcr = np.copy(srcr[z]) 
Tur = np.copy(tur[z])
Nsp = np.copy(nsp[z]) 

sim = simulation(Srcr, Tur, Nsp)

# if aslc == 'ASSL': paper_res = paper.assl
# elif aslc == 'AWSL': paper_res = paper.awsl

paper_res = paper.assl

corrections = sim.fuzzy_scale(paper_res)

SRCR[z] = sim.SRCR
TSUR[z] = sim.TUR
NSP[z] = sim.NSP
#####################################################################################